In [77]:
import numpy as np
import pandas as pd


In [78]:
iris =  pd.read_csv("/home/yatin/Downloads/IRIS.csv")
iris = iris.sample(frac=1).reset_index(drop=True) # Shuffle

In [79]:
# iris

In [80]:
X = iris[["sepal_length","sepal_width","petal_length","petal_width"]]
X = np.array(X)
# X.shape

In [81]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)

Y = iris.species
Y = one_hot_encoder.fit_transform(np.array(Y).reshape(-1, 1))
# Y

In [82]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20)
X_train=X_train.reshape(4,120)
Y_train=Y_train.reshape(3,120)
X_test=X_test.reshape(4,30)
Y_test=Y_test.reshape(3,30)
print("X_train.shape",X_train.shape,"\nY_train.shape",Y_train.shape,"\nX_test",X_test.shape,"\nY_test",Y_test.shape)

X_train.shape (4, 120) 
Y_train.shape (3, 120) 
X_test (4, 30) 
Y_test (3, 30)


In [83]:
def initialize_parameters(layers_dims, print_shapes = False):
    
    L = len(layers_dims)
    parameters ={}
    
    for i in range(1,L) :
        parameters['W'+ str(i)] = np.random.rand(layers_dims[i],layers_dims[i-1])*0.01
        parameters['b'+ str(i)] = np.zeros((layers_dims[i],1))
        
    if print_shapes :
        initialize_parameters_print_shapes(parameters)
                
    return parameters        

In [84]:
def initialize_parameters_print_shapes(parameters):
        print('W'+ str(i),parameters['W'+ str(i)].shape)
        print('b'+ str(i),parameters['b'+ str(i)].shape)

In [85]:
def relu(Z):
    A = np.maximum(Z,0,Z)
    return A

def sigmoid(Z):
    A = 1/(1+np.exp(-Z))
    return A

In [86]:
def forward_propogation(X,parameters,layers_dims):
    
    L = len(layers_dims)
    A_prev = X
    
    for i in range(1,L):
        parameters ['Z'+ str(i)] = np.dot(parameters ['W'+ str(i)],A_prev) + parameters ['b'+ str(i)]
        parameters ['A'+ str(i)] = relu(parameters ['Z'+ str(i)])
        A_prev = parameters ['A'+ str(i)]
    
    parameters ['AL'] = parameters.pop('A4') 
    
    return parameters ['AL'] ,parameters

In [87]:
# layers_dims = [3,5,4,2,1]
# parameters = initialize_parameters(layers_dims,print_par=False)
# X = np.random.rand(3,40)
# AL,cache = forward_propogation(X,parameters)
# print(cache)

In [88]:
def compute_cost(Y,AL):
    m = Y.shape[1]
    cost = (-1/m)*np.sum(np.multiply(Y,np.log(AL),np.multiply(1-Y,np.log(1-AL))))
    return(cost)

In [89]:
# Y = np.random.rand(2,2)
# AL = np.random.rand(2,2)
# cost=compute_cost(Y,AL)
# print(cost)

$\delta Z = (\frac{-Y}{AL} - \frac{1-Y}{1-AL})\times\sigma\prime(Z)  =  \delta A\times\sigma\prime(Z) $

In [90]:
def sigmoid_backwards(cache):
    return sigmoid(cache)*(1-sigmoid(cache))
    
def relu_backwards(Z,dA):
    dZ = np.array(dA, copy=True) 
    dZ[Z <= 0] = 0
    return dZ


$\large\delta A^{[l]} = (\frac{-Y}{AL} - \frac{1-Y}{1-AL})$

$\large\delta Z^{[l]} =  \delta A\times\sigma\prime(Z) $

$\large\delta W^{[l]} =  \frac {1}{m}\sum _{i=1}^{m}\delta Z\times A^{[l-1]T}$

$\large\delta b^{[l]} = \frac {1}{m}\sum _{i=1}^{m}\delta Z$

$\large\delta A^{[l-1]} = W^{[l]T} dZ^{[l]}$

In [91]:
def backpropogation(layers_dims,parameters,AL,Y,X,print_shapes = False):
    caches = {}
    m = Y.shape[1]
    L = len(layers_dims)
    flag = True
    parameters['A' + str(0)] = X
    for l in reversed(range(1,L)):
        
        if (flag) :   
            caches['dAL'] = np.divide(-Y,AL)+np.divide(1-Y,1-AL)                               
            caches['dZ'+ str(l)] = caches['dAL']*sigmoid_backwards(parameters['Z'+ str(l)])    
            caches['dW'+ str(l)] = (1/m)*np.dot(caches['dZ'+ str(l)],parameters['A' + str(l-1)].T)          
            caches['db'+ str(l)] = (1/m)*np.sum(caches['dZ'+ str(l)],axis=1,keepdims=True)     
            caches['dA' + str(l-1)] = np.dot(parameters['W'+ str(l)].T,caches['dZ' + str(l)])
            flag = False
        else:
            caches['dZ'+ str(l)] = caches['dA' + str(l)]*relu_backwards(parameters['Z'+ str(l)],caches['dA'+ str(l)])
            caches['dW'+ str(l)] = (1/m)*np.dot(caches['dZ'+ str(l)],parameters['A' + str(l-1)].T)
            caches['db'+ str(l)] = (1/m)*np.sum(caches['dZ'+ str(l)],axis=1,keepdims=True)
            if (l-1 != 0) :
                caches['dA' + str(l-1)] = np.dot(parameters['W' + str(l)].T,caches['dZ' + str(l)]) 
                
    if print_shapes :
        backpropogation_print_shapes(caches)
    
    return caches

In [92]:
def backpropogation_print_shapes(caches):
    print ('dAL',caches['dAL'].shape)
    for l in range(1,L):
        print("dZ"+ str(l),caches['dZ'+ str(l)].shape)
        print("dW"+ str(l),caches['dW'+ str(l)].shape)
        print("db"+ str(l),caches['db'+ str(l)].shape)
        if (l-1 != 0) :
            print('dA' + str(l-1),caches['dA' + str(l-1)].shape)

In [93]:
# layers_dims = [3,5,4,2,1]
# parameters = initialize_parameters(layers_dims,print_par=False)
# X = np.random.rand(3,40)
# Y = np.random.rand(1,40)
# AL,cache = forward_propogation(X,parameters)
# caches = backpropogation(layers_dims,cache,AL,Y,X)
# # print(caches)

$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]} \tag{16}$$
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]} \tag{17}$$

In [94]:
def update_parameters(parameters,caches,alpha,L):
    for l in range(L-1):
        parameters['W'+ str(l+1)] = parameters['W'+ str(l+1)] - alpha * caches['dW'+ str(l+1)]
        parameters['W'+ str(l+1)] = parameters['W'+ str(l+1)] - alpha * caches['dW'+ str(l+1)]
    return parameters

In [95]:
# cache_updated = update_parameters(cache,caches,0.01)
# # print(cache_updated)

In [96]:
def predict(X, Y, parameters,layers_dims):
    m = X.shape[1]
    n = len(layers_dims)  
    p = np.zeros(Y.shape)
    probas, caches = forward_propogation(X, parameters,layers_dims)
    
    _max = probas[0][0]  
    for i in range(0, probas.shape[1]):
        for j in range(0,probas.shape[0]):
            if (probas[i][j] > _max):
                p, _max = j,probas[i][j]
        for k in range(0,probas.shape[0]):   
            if (k == p) :
                probas[i][k] = 1
            else :
                probas[i][k] = 0
            
    print("Accuracy: "  + str(np.sum((p == Y)/m)))
        
    return p

In [100]:
def nn( X, Y, layers_dims, iterations,alpha, print_cost= False):
    parameters = initialize_parameters(layers_dims, print_shapes = False)
    costs = []
    for i in range(1,iterations):
        
        AL ,caches = forward_propogation(X,parameters,layers_dims)
        cost = compute_cost(Y,AL)
        caches = backpropogation(layers_dims,parameters,AL,Y,X,print_shapes = False)
        parameters = update_parameters(parameters,caches,alpha,len(layers_dims))
        
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)
    p = predict(X, Y, parameters,layers_dims)
    return p

In [101]:
layers_dims = [X_train.shape[0],5,4,2,Y_train.shape[0]]
p = nn( X_train, Y_train, layers_dims, 1500,0.01, print_cost= False)

/home/yatin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until
/home/yatin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in multiply
  This is separate from the ipykernel package so we can avoid doing imports until
/home/yatin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
/home/yatin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.
/home/yatin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in multiply
/home/yatin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: Ru

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()